In [1]:
%autosave 0

Autosave disabled


# Validação do Modelo
**O que você vai ver neste módulo?**:
- <a href="#1">Por que é necessário validar nosso modelo?</a>
- <a href="#2">Técnica 1: Extrair um pedaço da base de treino e usar como base de teste</a>
- <a href="#3">Técnica 2: Cross-validation</a>
- <a href="#4">Técnica 3: Confusion Matrix</a>
- <a href="#5">Técnica 4: Adversarial Validation</a>

## Por que é necessário validar nosso modelo? <a name='1' />

> Você não pode melhorar o que você não consegue medir

O objetivo é identificar como o nosso modelo se comporta, recebendo um feedback para saber se você está indo no caminho certo.

**Validação do seu modelo é algo que você deve se preocupar desde o início, antes mesmo de iniciar um processo mais profundo de análise dos dados.**

Vamos utilizar a base do Titanic novamente. Para facilitar o nosso trabalho, iremos utilizar uma base com as features já preparadas indo direto ao ponto.

In [2]:
import pandas as pd

# Carregando os dados de treinamento e de testes já limpos e preparados
train_df = pd.read_csv('../data/titanic/train_clean.csv')
test_df = pd.read_csv('../data/titanic/test_clean.csv')

In [3]:
train_df.head(3)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,FamilySize,IsAlone,FareCat
0,1,0,3,0,22,1,0,7.2500,1,1,2,0,1
1,2,1,1,1,38,1,0,71.2833,2,3,2,0,4
2,3,1,3,1,26,0,0,7.9250,1,2,1,1,2


In [4]:
test_df.head(3)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,FamilySize,IsAlone,FareCat
0,892,3,0,34,0,0,7.8292,3,1.0,1,1,1
1,893,3,1,47,1,0,7.0000,1,3.0,2,0,1
2,894,2,0,62,0,0,9.6875,3,1.0,1,1,2


In [5]:
# Vamos carregar as informacoes de treino e teste para 
X_train = train_df.drop(["Survived", "PassengerId"], axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()

# Criamos nosso RandomForestClassifier com 100 estimadores
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=0)
random_forest.fit(X_train, Y_train)

# Vamos fazer a validação do score utilizando a propria base de treinamento
random_forest.score(X_train, Y_train)

0.98204264870931535

Podemos ver que o score está bastante alto. Porém estamos validando com os mesmos dados utilizados para o modelo aprender, isso é o suficiente para que o modelo se saia bem todas as vezes.

Porém, este número nos diz alguma coisa? Será que nosso modelo está realmente bom o suficiente? Como será que ele se sairia com novos dados?

## Técnica 1: Extrair um pedaço da base de treino e usar como base de teste<a name='1' />

In [6]:
from sklearn.model_selection import ShuffleSplit

In [7]:
ss = ShuffleSplit(n_splits=1, test_size=0.20, random_state=0)
for index, (train_index, test_index) in enumerate(ss.split(X_train)):
    print("%s %s" % (train_index, test_index))

[140 439 817 378 491 331 588 358 674 162 580 402 671 875 727 428  54  45
 172 447 597 204 240 672  39 283 308  18 769 156 482 210 647  50 852 239
 646 122 788 167 306 379 742  10 105  68 253 320 706 271 424 752 615   2
 882 467 403 258 720 272 154 832 820 382  20 585  65 680 878 304 823 521
 333 698 481  71  49 573 338 261 116  76  48 843 263 718 620 786  64 857
  52 386 124 578 549 118  12 157 127 653 235 785 241 351 862 851 753 532
 485 695 475  17 476 533 416 345 242 344 170 187 800 457 652 451  78 889
 198 492 813 526 870  21 885 799 250 243 701  35  81 159 744 524 109 337
 443  92 364 434 465 731 876 211 811 165 238 188 471 553 456 366 592 738
 155 391 886 724 453  66 841 408 462 268 161 363 406 866 881 618 100 722
 678 229 334 558 669 807 520 816 220 810 309 688 787  15 245 409   6 452
 441 479 104 303 650 208 302  51  90 354 563  22 374 825 218 657 422 420
 460 205 529 190 361 735 375 185 194 401 436 583 132 233 173 178 313 315
 713 859 864 206 557 493  96 527 761 572  74  89  3

In [8]:
X_train.values[train_index]

array([[  3.,   1.,  21., ...,   3.,   0.,   3.],
       [  2.,   0.,  31., ...,   1.,   1.,   2.],
       [  2.,   0.,  31., ...,   3.,   0.,   4.],
       ..., 
       [  3.,   0.,  25., ...,   1.,   1.,   1.],
       [  3.,   1.,  36., ...,   2.,   0.,   3.],
       [  2.,   0.,  60., ...,   3.,   0.,   4.]])

In [9]:
ss = ShuffleSplit(n_splits=1, test_size=0.20, random_state=0)
for train_index, test_index in ss.split(X_train):
    # Treinar somente com os 80% da base de treinamento
    random_forest.fit(X_train.values[train_index], Y_train.values[train_index])

    # Extrair o score a partir dos outros 20%
    print(random_forest.score(X_train.values[test_index], Y_train.values[test_index]))

0.843575418994


## Técnica 2: Cross-Validation<a name='3' />

In [10]:
# Numero de vezes que iremos fazer o ciclo
folds = 5
final_score = 0

# Chamamos a funcao do SkLearn com o ShuffleSplit passando o numero de folds
ss = ShuffleSplit(n_splits=folds, test_size=0.20, random_state=0)
for train_index, test_index in ss.split(X_train):
    # Treinar somente com os 80% da base de treinamento
    random_forest.fit(X_train.values[train_index], Y_train.values[train_index])

    # Extrair o score a partir dos outros 20%
    score = random_forest.score(X_train.values[test_index], Y_train.values[test_index])
    print(score)
    
    final_score += score

# O score final será a media de todos os folds
final_score = final_score / folds
final_score

0.843575418994
0.759776536313
0.804469273743
0.787709497207
0.843575418994


0.80782122905027942

## Técnica 3: Confusion Matrix <a name='4' />

In [11]:
from sklearn.metrics import confusion_matrix

# Vamos pegar apenas 80% da base e usar os outros 20% como testes para ficar mais facil de exemplificar 
# (se usarmos tudo dá 100% de acerto)
ss = ShuffleSplit(n_splits=1, test_size=0.20, random_state=0)
for train_index, test_index in ss.split(X_train):
    random_forest.fit(X_train.values[train_index], Y_train.values[train_index])
    
    y_pred = random_forest.predict(X_train.values[test_index])
    y_true = Y_train.values[test_index]
    
    result = confusion_matrix(y_true, y_pred)

print('Positive/Positive (OK):', result[0][0])
print('Positive/Negativo (ER):', result[0][1])
print('Negativo/Positivo (ER):', result[1][0])
print('Negativo/Negativo (OK):', result[1][0])
result

Positive/Positive (OK): 100
Positive/Negativo (ER): 10
Negativo/Positivo (ER): 18
Negativo/Negativo (OK): 18


array([[100,  10],
       [ 18,  51]])

## Técnica 4: Adversarial Validation <a name='4' />

Esta técnica pode ser utilizada quando as bases disponíveis para testes é significativamente diferente da base disponível para treinamento. Em algumas situações isso pode acontecer.

Isso foi muito comentado em uma competição do Santander no Kaggle e também em uma outra competição que tem se tornado popular chamada <a href="http://www.numerai.com">Numerai</a>

Mas o que isso significa? Significa que os scores dos nossos modelos que temos no treinamento são significativamente diferentes dos scores quando nós subimos nossos resultados para os servidores. 

Em competições isso pode acontecer e no mundo real também! Então vale a pena gastarmos um tempo nesta técnica relativamente simples e que pode ajudar muito o nosso processo.

In [12]:
# Vamos sinalizar a origem dos dados (0 para dados de treinamento e 1 para dados de testes)
train_df['origin'] = 0
test_df['origin'] = 1

full_df = pd.concat((train_df.drop(["Survived", "PassengerId"], axis=1), test_df.drop("PassengerId", axis=1)))
print(len(train_df), len(test_df), len(full_df))

891 418 1309


Até agora, tudo bem... vamos ver como está a nossa nova base final para fazer o novo treinamento

In [13]:
full_df.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,FamilySize,IsAlone,FareCat,origin
0,3,0,22,1,0,7.2500,1,1.0,2,0,1,0
1,1,1,38,1,0,71.2833,2,3.0,2,0,4,0
2,3,1,26,0,0,7.9250,1,2.0,1,1,2,0
3,1,1,35,1,0,53.1000,1,3.0,2,0,4,0
4,3,0,35,0,0,8.0500,1,1.0,1,1,2,0


In [14]:
random_forest = RandomForestClassifier(n_estimators=100, random_state=0)
random_forest.fit(full_df.drop('origin', axis=1), full_df['origin'])

# Vamos usar o predict_proba que retorna a probabilidade de ser de origem de testes (mais proximo de 0 é treinamento, 1 é teste)
train_df['origin'] = random_forest.predict_proba(X_train)[:, 1]

train_df.head(3)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,FamilySize,IsAlone,FareCat,origin
0,1,0,3,0,22,1,0,7.2500,1,1,2,0,1,0.100000
1,2,1,1,1,38,1,0,71.2833,2,3,2,0,4,0.200000
2,3,1,3,1,26,0,0,7.9250,1,2,1,1,2,0.063333


In [15]:
# Vamos ordenar do menor para o maior (os que estiverem no final, foram confundidos com a base de testes)
i = train_df['origin'].argsort()
train_sorted = train_df.iloc[i]

# Vamos tirar os 200 ultimos itens e considerar eles como base de testes
validation_size = 200

# Pronto agora pegamos o X_train, Y_train e o X_test baseado nos ultimos itens que se confundem com a base de testes ou não
X_train = train_df.iloc[:-validation_size].drop(["Survived", "PassengerId"], axis=1)
Y_train = train_df.iloc[:-validation_size]["Survived"]
X_test  = train_df.iloc[-validation_size:].drop("PassengerId", axis=1).copy()